In [1]:
# the purpose of this notebook is to explore the separated 10m, 30m, and 50m datasets and their aggregate data (average
# pitch/roll, variance in turbulence, battery loss, all that jazz)

In [1]:
# import tings
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
# fix individual logs
# running the function(s) below, I found that Jan07.3-35_10m and Jan06.4-47_30m had mistakenly collected data on the way DOWN, too, which confounds
# the data. here, we reassign that file itself but without the distracting irrelevant data

badfile1 = pd.read_csv(Path('UsableLogs/Jan07.3-35_10m.csv'))
badfile1 = badfile1[badfile1['index'] <= 381]
badfile1.to_csv('UsableLogs/Jan07.3-35_10m.csv', index=False)

badfile2 = pd.read_csv(Path('UsableLogs/Jan06.4-47_30m.csv'))
badfile2 = badfile2[badfile2['index'] <= 603]
badfile2.to_csv('UsableLogs/Jan06.4-47_30m.csv', index=False)

In [3]:
# function that turns the files fed into it into a single row of aggregate data

# if file ends with _10m, entry in 'altitude' section is 10m, etc.
# include file name to check weird outliers out
# find average of all speeds, voltage, current, pitch, roll, battery_percent lost over time
# each time, append the row to the dataframe previously made

directory = Path('UsableLogs')
preaggregate_logs = directory.glob('*.csv')
output_dir = Path('AggregateLogs')

def process_usable_logs():
    aggregated_data = []

    for file_path in preaggregate_logs:        
        # Parse altitude from filename
        altitude = None
        if '_10m' in file_path.name:
            altitude = 10
        elif '_30m' in file_path.name:
            altitude = 30
        elif '_50m' in file_path.name:
            altitude = 50
        
        try:
            # Read the CSV
            df = pd.read_csv(file_path)
            start_time, end_time = pd.to_datetime(df['timestamp'].iloc[0]), pd.to_datetime(df['timestamp'].iloc[-1])
            flight_time = (end_time - start_time).total_seconds()
            
            # calculate the aggregate stats
            stats = {
                'file_name': file_path.stem,
                'flight_time(seconds)': flight_time,
                'altitude': altitude,
                'x_speed_avg': df['xSpeed(mph)'].mean(),
                'y_speed_avg': df['ySpeed(mph)'].mean(),
                'z_speed_avg': df['zSpeed(mph)'].mean(),
                'speed_avg': df['speed(mph)'].mean(),
                'speed_sd': df['speed(mph)'].std(),
                'voltage_avg': df['voltage(v)'].mean(),
                'voltage_sd': df['voltage(v)'].std(),
                'current_avg': df['current(A)'].mean(),
                'current_sd': df['current(A)'].std(),
                'pitch_avg': df['pitch(degrees)'].mean(),
                'pitch_sd': df['pitch(degrees)'].std(),
                'roll_avg': df['roll(degrees)'].mean(),
                'roll_sd': df['roll(degrees)'].std(),
                # We just take battery percent from first row minus last row
                'battery_loss_per_second': (df['battery_percent'].iloc[0] - df['battery_percent'].iloc[-1]) / flight_time
            }
            
            aggregated_data.append(stats)
            
        except Exception as e:
            print(f"Error processing {file_path.name}: {e}")

    return pd.DataFrame(aggregated_data).sort_values('file_name')
aggregate_df = process_usable_logs()

In [4]:
aggregate_df

,file_name,flight_time(seconds),altitude,x_speed_avg,y_speed_avg,z_speed_avg,speed_avg,speed_sd,voltage_avg,voltage_sd,current_avg,current_sd,pitch_avg,pitch_sd,roll_avg,roll_sd,battery_loss_per_second
19,Jan06.4-37_10m,12.0,10,0.000000,0.000000e+00,0.000000,0.000000,0.000000,7.493718,0.014779,4.726492,0.171985,2.045968,0.917792,-0.608871,0.666506,0.083333
29,Jan06.4-37_30m,19.0,30,-0.001236,-1.112291e-02,0.019774,0.022987,0.072540,7.482028,0.022114,4.535116,0.315259,4.633149,1.494622,-4.328729,0.886474,0.052632
16,Jan06.4-37_50m,19.0,50,0.012092,0.000000e+00,-0.008464,0.012092,0.050720,7.417032,0.012993,4.973200,0.121762,1.442162,1.403588,-5.104324,0.743145,0.052632
116,Jan06.4-41_10m,16.0,10,0.000000,-2.663024e-03,0.000000,0.002663,0.024334,7.348792,0.022183,5.007988,0.285532,4.276190,1.347530,-1.771429,0.676212,0.062500
117,Jan06.4-41_30m,18.0,30,0.002528,0.000000e+00,0.000000,0.002528,0.023711,7.326424,0.018667,4.748463,0.316471,4.555932,0.608072,-4.139548,0.746747,0.055556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,Jan13.12-24_30m,23.0,30,-0.011374,0.000000e+00,0.001896,0.013270,0.052955,7.618220,0.011123,4.538564,0.220859,5.229661,1.468553,-4.361017,0.593731,0.043478
42,Jan13.12-24_50m,25.0,50,-0.009321,-9.320583e-04,0.000932,0.014913,0.055916,7.553808,0.029723,5.025162,0.514583,7.666667,1.211475,-10.365000,1.379233,0.080000
61,Jan13.12-28_10m,25.0,10,0.006444,2.284410e-19,-0.009206,0.058731,0.108534,7.501407,0.020691,5.231523,0.273290,2.323045,1.852038,-5.375309,2.509545,0.080000
78,Jan13.12-28_30m,23.0,30,-0.002969,5.938779e-03,0.001980,0.043749,0.095763,7.427527,0.027382,5.494327,0.777363,5.873894,2.000773,-11.090265,2.037579,0.043478


In [6]:
# push to csv
aggregate_df.to_csv('aggregate_data_TEST.csv', index=False) 